In [5]:
import cv2
import sys,os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from feature_extraction_pipeline import *

#### Loop through all folders(`game_id`) and locate the CTA button from the end_frame captured from the video

In [ ]:
import pandas as pd
rootdir = '../data/Challenge_Data/Assets/'
max_dir=10
dir_count=0
cta_counter = 0
max_loop=1
loop=0
no_cta_game_ids = []
for subdir, dirs, files in os.walk(rootdir):
    loop+=1
    if loop > max_loop:
        break # In order just to complete only one walkthrough
    for dir in dirs:
        # dir_count+=1
        # if dir_count > max_dir:
        #     break
        dir_path = os.path.join(subdir, dir)
        # dir_path = "../data/Challenge_Data/Assets/942616462420cb3890d00c65fc644e73"
        cta_loc_dict = {}
        for _,d, contents in os.walk(dir_path):
            # print(len(contents))
            # p.append(contents)
            cta_file_name = ""
            for content in contents:
                # Loop through all the contents 
                if (str(content))[-3:] == "png" or (str(content))[-3:] == "jpg":
                    # check if the current content is cta file
                    content_name = str(content).split(".")
                    if content_name[0][-3:] == "cta":
                        cta_file_name = content
                        break
                    else:
                        continue
                    # p.append(content)
                    # print(colors_x)
                    # break
                # print(content)
            end_frame_dir = "../data/end_beg_frames/end_frame/"
            for _, _, end_frame_files in os.walk(end_frame_dir):
                # print(f"LEN:{len(end_frame_files)}")
                for f in end_frame_files:
                    if dir in str(f):
                        if cta_file_name!="":
                            try:
                                vals = locate_image_on_image(f"{dir_path}/{cta_file_name}", f"{end_frame_dir}/{f}")
                                cta_loc_dict = {
                                    "game_id":dir,
                                    "cta_start_x":vals[2][0],
                                    "cta_start_y":vals[2][1],
                                    "cta_end_x":vals[3][0],
                                    "cta_end_y":vals[3][1],
                                    "cta_height":vals[0],
                                    "cta_width":vals[1]
                                }
                                # print("$$$$$$$$")
                                # print(vals)
                            except Exception as e:
                                continue
                                print(str(e))
                        break
                    else:
                        continue
                break
            if cta_loc_dict:
                break
            for content in contents:
                if (str(content))[-3:] == "png" or (str(content))[-3:] == "jpg":
                    # print(content)
                    if "endframe" in str(content):
                        # print("FOUND")
                        if cta_file_name!="":
                            # print("FILENAME")
                            # print(cta_file_name)
                            # print(content)
                            try:
                                vals = locate_image_on_image(f"{dir_path}/{cta_file_name}", f"{dir_path}/{content}")
                                if vals is not None:
                                    cta_loc_dict = {
                                        "game_id":dir,
                                        "cta_start_x":vals[2][0],
                                        "cta_start_y":vals[2][1],
                                        "cta_end_x":vals[3][0],
                                        "cta_end_y":vals[3][1],
                                        "cta_height":vals[0],
                                        "cta_width":vals[1]
                                    }
                                    break
                                # print("$$$$$$$$")
                                # print(vals)
                            except Exception as e:
                                print("HEREEE")
                                continue
                                print(str(e))
                        else: break
                    # else:
                    #     print("DM")
            break
        
        # print(dir_path)
        # print(cta_loc_dict)
        if cta_loc_dict:
            cta_counter+=1
            # print("SAVING")
            df = pd.DataFrame(
                cta_loc_dict, index=[0]
            )
            # print(df.shape)   
            df.to_csv(f"{dir_path}/cta_position.csv")
        # print(df.shape)
        else:
            no_cta_game_ids.append(dir)
            df = pd.DataFrame({
                "game_id":dir,
                "cta_start_x":"",
                "cta_start_y":"",
                "cta_end_x":"",
                "cta_end_y":"",
                "cta_height":"",
                "cta_width":""
            },index=[0])
            df.to_csv(f"{dir_path}/cta_position.csv")
        # break
print(loop)
print(cta_counter)
print(len(no_cta_game_ids))

In [14]:
df = pd.DataFrame.from_dict({"game_id":no_cta_game_ids}).to_csv("../data/no_cta.csv")